In [1]:
from dataloader_utils import get_conbined_permute_mnist
from autoencoder import Autoencoder
from autoencoder_utils import * 

import numpy as np


In [92]:
RANDOM_SEED = np.random.randint(100)
#RANDOM_SEED = 42

NUM_TASK = 20
BATCH_SIZE = 120


OUTLIER_THRESHOLD = 0.1*BATCH_SIZE
NEW_AUTOENCODER_EPOCH = 800
TRAIN_AUTOENCODER_EPOCH = 100
CODE_DIM = 300

In [ ]:
train_loader, test_loader = get_conbined_permute_mnist(NUM_TASK, BATCH_SIZE, RANDOM_SEED)

In [ ]:
auto_list = {}
expert_list = {}

#debug
record = {}

for i, data in enumerate(train_loader):
    images, labels, indicies = data
    
    #initial 
    if len(auto_list)==0:
        #debug
        print(f"[new @ batch {i}] autoencoder at {len(auto_list)} for index: {indicies[0].item()}")
        record[indicies[0].item()] = len(auto_list)
        
        #initial autoencoder
        new_autoencoder = Autoencoder(input_dims=28*28, code_dims=CODE_DIM)
        for epoch in range(NEW_AUTOENCODER_EPOCH):
            new_autoencoder.optimize_params(images, images)
        auto_list[len(auto_list)] = new_autoencoder
        
        #to-do add initial expert
        
        continue
    
    #find best autoencoder
    best_index = find_best_autoencoders(images, auto_list)
    best_autoencoder = auto_list[best_index]
    
    #calculate outliers
    outliers = find_num_of_outliers(images, best_autoencoder)
    #print(f"outliers for best autoencoders {best_index}: {outliers}")
    
    if outliers > OUTLIER_THRESHOLD:
        #debug
        print(f"[new @ batch {i}] autoencoder at {len(auto_list)} for index: {indicies[0].item()}")
        if indicies[0].item() in record:
            print(f"[error @ batch {i}] autoencoder exists but initialized a new one")
        else:
            record[indicies[0].item()] = len(auto_list)
        
        #add new autoencoder
        best_autoencoder = Autoencoder(input_dims=28*28, code_dims=CODE_DIM)
        for epoch in range(NEW_AUTOENCODER_EPOCH):
            best_autoencoder.optimize_params(images, images)
        auto_list[len(auto_list)] = best_autoencoder
        
        #to-do add new expert
        
    else:
        #debug
        #print(f"training autoencoder at {best_index} with index: {indicies[0].item()}")
        if not indicies[0].item() in record or record[indicies[0].item()] != best_index:
            print(f"[error @ batch {i}] training autoencoder at {best_index} with index: {indicies[0].item()}")
        
        #train best autoencoder
        for epoch in range(TRAIN_AUTOENCODER_EPOCH):
            best_autoencoder.optimize_params(images, images)
        
        #to-do train exsisting expert